# GEC Bert - Inference

## Imports and Setup

In [4]:
%load_ext autoreload
%autoreload 2

In [37]:
# !pip install wandb

In [15]:
import pandas as pd
import pickle
import os
from utils import Config, Training_config
import wandb
import torch

print(torch.__version__)
main_args = Config()
# !mkdir cfgs
# !mkdir processed_data

1.12.1


In [39]:
# !pip install simpletransformers

In [6]:
from simpletransformers.ner import NERModel

In [43]:
args = Training_config(main_args.ECC_TRAIN_CONFIG)
labels = args.BIO_labels


## Inference with Local Model

In [44]:
reloaded = NERModel(model_type=args.model,model_name='outputs/gec_cows_l2h_small', labels=labels, use_cuda=False, ignore_mismatched_sizes=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [46]:
reloaded.predict(["Voy a ver un montón de gente."])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'Voy': 'O'},
   {'a': 'O'},
   {'ver': 'O'},
   {'un': 'O'},
   {'montón': 'O'},
   {'de': 'O'},
   {'gente.': 'O'}]],
 [[{'Voy': [[1.0854763,
      -0.316451,
      0.047038365,
      -0.34436443,
      0.2487065,
      -0.68969965,
      0.116881415,
      -0.21325706,
      -0.16128615,
      -0.4065132,
      2.1229296]]},
   {'a': [[0.6385935,
      -0.39191735,
      -0.12966174,
      -0.4718264,
      0.030699767,
      -1.0556124,
      0.11838204,
      -0.33300042,
      -0.062101148,
      -0.37952453,
      1.8581315]]},
   {'ver': [[1.3659866,
      -0.06448756,
      0.4369618,
      -0.781453,
      0.16887324,
      -0.48044986,
      0.55615455,
      -0.24134833,
      -0.47410864,
      -0.859541,
      2.0705187]]},
   {'un': [[1.1931605,
      -0.15088548,
      0.2279812,
      -0.61342067,
      -0.05016152,
      -0.78995043,
      0.26713532,
      -0.39121208,
      -0.5605346,
      -0.78983647,
      2.524552]]},
   {'montón': [[1.167207,
      0.027674

## Pickle the Model

In [48]:
with open('outputs/gec_cows_l2h_small.pkl', 'wb') as f:  # open a text file
    pickle.dump(reloaded, f) # serialize the list

## Push model to HuggingFace

In [1]:
!pip install huggingface_hub

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [32]:
with open('outputs/gec_cows_l2h_small.pkl', 'rb') as f:  # open a text file
    reloaded = pickle.load(f) # serialize the list
    print(reloaded.predict(['Hola que tal'])[0])
    torch.save(reloaded.state_dict(), "outputs/tt.pth")
    # reloaded.save_model('outputs/tt', model=reloaded)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Hola': 'O'}, {'que': 'O'}, {'tal': 'O'}]]


AttributeError: 'NERModel' object has no attribute 'state_dict'

In [31]:
instance_types = ['ml.r6g.xlarge', 'ml.c5.4xlarge', 'ml.c5.2xlarge', 'ml.c7g.2xlarge']

from sagemaker import image_uris
for cur in instance_types:
    image = image_uris.retrieve(
        framework="",
        #framework='pytorch',
        region='us-west-2',
        version='1.12',
        py_version='py38',
        image_scope='inference',
        instance_type=cur)

    print(f'{cur}:{image}')

ValueError: Unsupported framework: . Supported framework(s) for Graviton instances: {'xgboost', 'sklearn', 'tensorflow', 'pytorch'}.

## Access model artefact from WandB

In [8]:
run = wandb.init(project='gec_ner_expect', job_type="inference")
# use the latest version of the model

artifact = run.use_artifact('langbot/gec_ner_expect/run-47ehfc2y-history:v0', type='wandb-history')
artifact_dir = artifact.download()

print("model: ", artifact_dir)


wandb:   1 of 1 files downloaded.  


model:  ./artifacts/run-47ehfc2y-history:v0
